In [1]:
import pandas as pd
import scipy
import numpy as np
# import h5py

from constants import HARD_WIRED_GENOME_A_MATRIX_PATH, STRING_EXTRACTED_URL_PATH, STRING_PROTEIN_GENE_PATH, DATA_DIRECTORY, HURI_UNIQUE_GENE_SET, HARD_WIRED_GENOME_C_EDGELIST, HARD_WIRED_GENOME_B_EDGELIST, HARD_WIRED_GENOME_A_EDGELIST
# from mapping import get_unique_genes, generate_unique_gene_set_Huri

In [2]:
import matplotlib.pyplot as plt

import scanpy as sc
import networkx as nx

from scipy.linalg import eigvalsh

from anndata import AnnData

In [3]:

from constants import HARD_WIRED_GENOME_A_EDGELIST, HARD_WIRED_GENOME_B_EDGELIST, HARD_WIRED_GENOME_C_EDGELIST

from mapping import get_string_protein_gene_map, generate_string_mappings, generate_unique_gene_set_Huri, get_unique_genes, construct_HardWiredGenome_A_Matrix, construct_HardWiredGenome_B_Matrix, get_TF_set, construct_HardWiredGenome_A_Matrix

import os

from mapping import get_sorted_gene_order

flat_col_order = get_sorted_gene_order()

nodes = flat_col_order
len(nodes)


A_matrix, B_matrix, C_matrix = construct_HardWiredGenome_A_Matrix(threshold=0, nodes=nodes)

A_matrix_edgelist = A_matrix

ordering 18582 genes
Constructing Hard Wired Genome A Matrix



KeyboardInterrupt



In [ ]:
# from mapping import get_string_protein_gene_map, generate_string_mappings, generate_unique_gene_set_Huri, get_unique_genes, construct_HardWiredGenome_A_Matrix, construct_HardWiredGenome_B_Matrix

In [ ]:
# threshold = 150
# # path = HARD_WIRED_GENOME_A_MATRIX_PATH.strip('.csv') + f"_{threshold}.csv"

# path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}.csv"

# A_matrix_edgelist = pd.read_csv(path, index_col=0)

In [20]:
from network_utils import generate_gene_id_name_map, get_master_regulator_list

gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()


Extracted 43945 mappings from 73467 ensembl lines
Extracted 38606 mappings from gene id gtf file
Total Mappings Extracted 59979


In [24]:
from mapping import load_master_regulators, get_master_regulator_list, get_TF_lists

master_regulators_new = load_master_regulators()
master_regulator_list = get_master_regulator_list()

set(master_regulators_new).intersection(set(master_regulator_list))

len(set(master_regulators_new).union(set(master_regulator_list)))

Extracted 43945 mappings from 73467 ensembl lines
Extracted 38606 mappings from gene id gtf file
Total Mappings Extracted 59979
Number of Master regulators: 617
total master regulators from Old list: 439


688

In [33]:
len(set(master_regulators_new).intersection(set(master_regulator_list)))

367

In [15]:
from mapping import load_master_regulators, get_master_regulator_list, get_TF_lists
repressorlist, activatorlist, conflictedlist, tf_list = get_TF_lists()

print(f'Total TFs identified: {len(set(tf_list).union(set(master_regulators_new)))}')

 Total Activators : 174
Total Reprossors : 705
Total Conflicted : 729
Total Transcription Factors : 1608
Total TFs identified: 1857


In [11]:

from constants import HUMAN_TF_ACTIVITY_PATH

tflist = get_TF_set()
TF_activity_TRRUST = pd.read_csv(HUMAN_TF_ACTIVITY_PATH, sep='\t', header=None)

activity_scores = TF_activity_TRRUST.groupby(0)[2].value_counts().unstack(fill_value=0)
print(activity_scores)

print(TF_activity_TRRUST)


log = True

if log:
    print(f' Total Activators : {len(activatorlist)}')
    print(f'Total Reprossors : {len(repressorlist)}')
    print(f'Total Conflicted : {len(conflictedlist)}')

    print(f'Total Transcription Factors : {len(tflist)}')

processed 1639 Transcription Factors
2       Activation  Repression  Unknown
0                                      
AATF             1           1        3
ABL1             8           3        1
AES              2           0        1
AHR              8           2       16
AIP              1           4        0
...            ...         ...      ...
ZNF410           0           0        1
ZNF423           0           0        1
ZNF444           1           0        0
ZNF76            1           1        1
ZNRD1            2           0        0

[795 rows x 3 columns]
          0       1           2                  3
0      AATF     BAX  Repression           22909821
1      AATF  CDKN1A     Unknown           17157788
2      AATF    KLK3     Unknown           23146908
3      AATF     MYC  Activation           20549547
4      AATF    TP53     Unknown           17157788
...     ...     ...         ...                ...
9391  ZNF76  CDKN1A  Repression           15280358
9392  ZNF76

In [31]:
print(f"Total Master Regulators : {len(set(master_regulators_new).intersection(set(genes)))}")
print(f"Total intersection with Transcription Factors {len(set(master_regulators_new).intersection(set(tflist)))}")


Total Master Regulators : 560
Total intersection with Transcription Factors 367


In [38]:
import h5py
import pandas as pd
import numpy as np
from anndata import AnnData
import scanpy as sc

RNASEQ_PATH = '/scratch/indikar_root/indikar1/shared_data/HWG/data/RNAseq'

# mat_file = h5py.File('/Users/ramprakash/development/lab_projects/Rajapakse_lab/data/HWG/RNAseq/RNAseq Vectors 2015-2024.mat', 'r')
# print(mat_file)

X = pd.read_csv(f'{RNASEQ_PATH}/X.csv')
X_arr = X.to_numpy()
obs = pd.read_csv(f'{RNASEQ_PATH}/obs.csv')
var = pd.read_csv(f'{RNASEQ_PATH}/var.csv')



fibroblast_expression = X_arr[7]
fibroblast_expression = np.nan_to_num(fibroblast_expression, nan=0)
esc_expression = X_arr[11]
esc_expression = np.nan_to_num(esc_expression, nan=0)
myotube_expression = X_arr[12]
myotube_expression = np.nan_to_num(myotube_expression, nan=0)

fibroblast_nz_indices = np.nonzero(fibroblast_expression)
esc_nz_indices = np.nonzero(esc_expression)
myotube_nz_indices = np.nonzero(myotube_expression)

obs = obs.iloc[[7, 11, 12]]
samples = ['Fibroblast', 'EmbryonicStemCells', 'Myotube']


obs['CellType'] = samples

X = np.vstack([fibroblast_expression, esc_expression, myotube_expression])

adata = AnnData(X=X)

adata.var = var        # columns (genes)
adata.var_names = var['GeneStableID'].astype(str)
adata.var = adata.var.drop(columns=['GeneStableID'])

adata.obs = obs
adata.obs_names = adata.obs["CellType"].astype(str)
adata.obs = adata.obs.drop(columns=['CellType'])


sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

adata.write('./data/Genexpression.h5ad')

/home/nuvi/.local/lib/python3.11/site-packages/anndata/_core/anndata.py:740: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [7, 11, 12]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [24]:
from mapping import generate_gene_id_name_map 

gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()

Extracted 43945 mappings from 73467 ensembl lines
Extracted 38606 mappings from gene id gtf file
Total Mappings Extracted 59979


In [152]:
marker_gene_df = pd.read_csv('./data/clean_marker_genes.csv')

fibroblast_markers = marker_gene_df[marker_gene_df['cell_type'] == 'fibroblasts']
print(fibroblast_markers)
marker_genes = list(fibroblast_markers['gene_name'])

gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()

marker_ids = [ gene_name_id_map[i] for i in marker_genes ]

expr_values = adata.X[adata.obs_names == 'Fibroblast'].flatten()

expr_series = pd.Series(expr_values, index=adata.var_names)
marker_expr = expr_series[expr_series.index.isin(marker_ids)]
print(len(marker_expr))


# threshold = marker_expr.mean()
threshold = marker_expr.median()
high_expr = expr_series[expr_series > threshold]

top_genes = high_expr.sort_values(ascending=False)

top_genes = top_genes[top_genes.index.notna()]

fibroblast_expressed_genes = top_genes.index.tolist()


    gene_name    cell_type  is_tf   source    category
697   RARRES2  fibroblasts  False  panglao  fibroblast
698     CELA1  fibroblasts  False  panglao  fibroblast
699       LUM  fibroblasts  False  panglao  fibroblast
700     PRRX1  fibroblasts   True  panglao  fibroblast
701    SCARA5  fibroblasts  False  panglao  fibroblast
..        ...          ...    ...      ...         ...
924    COL8A1  fibroblasts  False  panglao  fibroblast
925      PTX3  fibroblasts  False  panglao  fibroblast
926     CEBPB  fibroblasts   True  panglao  fibroblast
927      PKD2  fibroblasts  False  panglao  fibroblast
928    CTHRC1  fibroblasts  False  panglao  fibroblast

[232 rows x 5 columns]
230


In [78]:
fibroblast_filtered_edgelist = A_matrix_edgelist[
    A_matrix_edgelist['source'].isin(fibroblast_expressed_genes) & A_matrix_edgelist['target'].isin(fibroblast_expressed_genes)
]

current_edgelist = fibroblast_filtered_edgelist

start_index = 0
end_index = 18581

# ENSG00000129152 - MYOD1
perturbed_edges = ['ENSG00000129152']
move_list = ['ENSG00000129152']

# filtered_edgelist, filtered_nodes = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)
filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)
# filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=False, move_list=move_list)


/tmp/ipykernel_2053901/452622973.py:66: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ts_repression_edges = concat_df[
/tmp/ipykernel_2053901/452622973.py:70: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  st_repression_edges = concat_df[


In [24]:
gene_names = adata.var_names
cell_names = adata.obs_names

gen_threshold = 0.2

X_dense = adata.X.toarray() if not isinstance(adata.X, np.ndarray) else adata.X


expressed_genes_per_cell = []

for i, row in enumerate(X_dense):
    expressed = gene_names[row > gen_threshold]
    expressed_genes_per_cell.append(list(expressed))

# Optional: convert to DataFrame for easier inspection
df = pd.DataFrame({
    'cell': cell_names,
    'expressed_genes': expressed_genes_per_cell
})

print("Number of Fibroblast expressed genes", len(df.iloc[0].expressed_genes))
print("Number of ESC expressed genes", len(df.iloc[1].expressed_genes))
print("Number of Myotube expressed genes", len(df.iloc[2].expressed_genes))

fibroblast_expressed_genes = df.iloc[0].expressed_genes
esc_expressed_genes = df.iloc[1].expressed_genes
myotube_expressed_genes = df.iloc[2].expressed_genes

Number of Fibroblast expressed genes 5006
Number of ESC expressed genes 7018
Number of Myotube expressed genes 7478


In [22]:
filtered_A_matrix_edgelist = A_matrix_edgelist[
    A_matrix_edgelist['source'].isin(flat_col_order) & A_matrix_edgelist['target'].isin(flat_col_order)
]

fil_path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"filetered_{threshold}.csv"

filtered_A_matrix_edgelist.to_csv(fil_path, index=False)


A_matrix_edgelist = filtered_A_matrix_edgelist

In [23]:
# fibroblast_expressed_genes
# myotube_expressed_genes
# esc_expressed_genes

fibroblast_filtered_edgelist = A_matrix_edgelist[
    A_matrix_edgelist['source'].isin(fibroblast_expressed_genes) & A_matrix_edgelist['target'].isin(fibroblast_expressed_genes)
]
myotube_filtered_edgelist = A_matrix_edgelist[
    A_matrix_edgelist['source'].isin(myotube_expressed_genes) & A_matrix_edgelist['target'].isin(myotube_expressed_genes)
]
esc_filtered_edgelist = A_matrix_edgelist[
    A_matrix_edgelist['source'].isin(esc_expressed_genes) & A_matrix_edgelist['target'].isin(esc_expressed_genes)
]


In [25]:
celltype = 'Fibroblast'
threshold = 150


# path = HARD_WIRED_GENOME_A_MATRIX_PATH.strip('.csv') + f"_{threshold}.csv"

path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}_{celltype}_{gen_threshold}.csv"

# fibroblast_filtered_edgelist.to_csv(path, index=False)
fibroblast_filtered_edgelist = pd.read_csv(path)



celltype = 'Myotube'
threshold = 150

path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}_{celltype}_{gen_threshold}.csv"

# df = pd.read_csv(path, index_col=0)
# myotube_filtered_edgelist.to_csv(path, index=False)

myotube_filtered_edgelist = pd.read_csv(path)



celltype = 'EmbryonicStemCells'
threshold = 150

path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}_{celltype}_{gen_threshold}.csv"

# df = pd.read_csv(path, index_col=0)
# esc_filtered_edgelist.to_csv(path, index=False)

esc_filtered_edgelist = pd.read_csv(path)


In [67]:

HARD_WIRED_GENOME_BP_MATRIX = f"{DATA_DIRECTORY}/HWG/BP_Matrix.csv"

c_matrix = pd.read_csv(HARD_WIRED_GENOME_C_EDGELIST)

master_regulators = c_matrix[c_matrix['source'] == c_matrix['target']]

master_regulator_list = list(c_matrix['source'])

transcription_factor_matx = c_matrix[c_matrix['source'] != c_matrix['target']]

transcription_factor_list = pd.unique(transcription_factor_matx[['source', 'target']].values.ravel())

BP_matrix = pd.read_csv(HARD_WIRED_GENOME_BP_MATRIX, index_col=0)

column_means = BP_matrix.mean()

sign_dict = column_means.apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0).to_dict()

gene_id_name_map, name_id_map = generate_gene_id_name_map()

# POU5F1, SOX2, KLF4, MYC
gene_id_name_map.update(dict(zip( ['ENSG00000134323', 'ENSG00000111704', 'ENSG00000204531', 'ENSG00000129152', 'ENSG00000136826', 'ENSG00000136997', 'ENSG00000181449', 'ENSG00000122180'], ['MYCN', 'NANOG' ,'POU5F1', 'MYOD1', 'KLF4', 'MYC', 'SOX2', 'MYOG'])))

# gene_id_name_map

def get_piece_network(edgelist, full_edgelist, start_index, end_index, order, perturbed_edges=[]):

    global sign_dict

    list_chunk = order[start_index:end_index]

    normal_edges = []
    activator_edges = []
    repressor_edges = []
    for edge in perturbed_edges:
        sign = sign_dict.get(edge, 0)
        if sign == -1:
            repressor_edges.append(edge)
        elif sign == 1:
            activator_edges.append(edge)
        else:
            normal_edges.append(edge)


    # get only the activators from the full edgelist

    ts_perturbed_edgelist = full_edgelist[
            full_edgelist['source'].isin(activator_edges) & full_edgelist['target'].isin(list_chunk)
    ]

    st_perturbed_edgelist = full_edgelist[
            full_edgelist['source'].isin(list_chunk) & full_edgelist['target'].isin(activator_edges)
    ]

    perturbed_edgelist = pd.concat([ts_perturbed_edgelist, st_perturbed_edgelist], ignore_index=True)
    

    filtered_edgelist = edgelist[
            edgelist['source'].isin(list_chunk) & edgelist['target'].isin(list_chunk)
    ]

    concat_df = pd.concat([perturbed_edgelist, filtered_edgelist], ignore_index=True)


    
    ts_repression_edges = concat_df[
        concat_df['source'].isin(repressor_edges) & filtered_edgelist['target'].isin(activator_edges)
    ]
    
    st_repression_edges = concat_df[
        concat_df['source'].isin(activator_edges) & filtered_edgelist['target'].isin(repressor_edges)
    ]

    repression_edgelist = pd.concat([ts_repression_edges, st_repression_edges], ignore_index=True)

    concat_df = concat_df[~concat_df['source'].isin(repressor_edges)]
    concat_df = concat_df[~concat_df['target'].isin(repressor_edges)]

    fin_df = pd.concat([concat_df, repression_edgelist], ignore_index=True)

    return fin_df, list_chunk + perturbed_edges, normal_edges, activator_edges, repressor_edges

In [ ]:


def construct_graph(filtered_edgelist, filtered_nodes, perturbed_edges, normal_edges, activator_edges, repressor_edges, node_attrs, time_based=False, move_list=[]):
    global BP_matrix
    global transcription_factor_list
    global master_regulator_list
    global gene_id_name_map
    column_means = BP_matrix.mean()

    sign_dict = column_means.apply(lambda x: 1 if x >= 0 else -1 if x < 0 else 0).to_dict()


    highlight_color = "#a345f0"
    repressor_color = "#d77132"
    initiator_color = "#f83e56"

    # g1_color = '#0096c7'
    # g2_color = '#00b4d8'
    # g3_color = "#90e0ef"
    g1_color = "#6b7fa6"
    g2_color = "#86c9d7"
    g3_color = "#c6d6d8"
    filtered_graph = nx.from_pandas_edgelist(filtered_edgelist, 'source', 'target', edge_attr='weight', create_using=nx.Graph())
    filtered_graph.add_nodes_from(filtered_nodes)
    filtered_graph.add_nodes_from(activator_edges)
    filtered_graph.add_nodes_from(repressor_edges)
    filtered_graph.add_nodes_from(move_list)



    # filtered_graph = connect_unconnected_nodes(filtered_graph)


    centrality = nx.degree_centrality(filtered_graph)
    cent_values = np.array(list(centrality.values()))
    cent_min, cent_max = cent_values.min(), cent_values.max()
    norm_centrality = {node: (val - cent_min) / (cent_max - cent_min) for node, val in centrality.items()}

    # Use a colormap (e.g., 'viridis') to map centrality → color
    # cmap = cm.get_cmap('winter')
    cmap = custom_cmap
    node_colors = {node: cmap(norm_centrality[node]) for node in filtered_graph.nodes()}

    degree_centrality = nx.degree_centrality(filtered_graph)


    # for dynamic graphs
    timepoints = len(perturbed_edges) + 1

    for node in node_attrs:
        key = node['key']
        nat = node['attributes']
        x = nat['x']
        y = nat['y']
        if key in filtered_graph:
            # degree = degree_centrality[key]
            degree = filtered_graph.degree[key]
            filtered_graph.nodes[key]['size'] = 15 + 1 * degree

            if abs(x) > 3000 or abs(y) > 3000:
                filtered_graph.nodes[key]['x'] = 0
                filtered_graph.nodes[key]['y'] = 0

            filtered_graph.nodes[key]['x'] = x
            filtered_graph.nodes[key]['y'] = y
            if time_based:
                filtered_graph.nodes[key]['color_dynamic'] = f"<[{0},{timepoints}{'#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])}]>"
                filtered_graph.nodes[key]['perturbed_dynamic'] = f"<[{0},{timepoints}{'#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])}]>"
                filtered_graph.nodes[key]['color'] = '#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])
            else:
                filtered_graph.nodes[key]['color'] = '#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])

            if key in master_regulator_list:
                filtered_graph.nodes[key]['color'] = g1_color
                filtered_graph.nodes[key]['type'] = 'g1'
            elif key in transcription_factor_list:
                filtered_graph.nodes[key]['color'] = g2_color
                filtered_graph.nodes[key]['type'] = 'g2'
            else:
                filtered_graph.nodes[key]['color'] = g3_color
                filtered_graph.nodes[key]['type'] = 'g3'



            filtered_graph.nodes[key]['genename'] = gene_id_name_map.get(key, "NA")
            filtered_graph.nodes[key]['start'] = 0
            filtered_graph.nodes[key]['end'] = timepoints + 10
            filtered_graph.nodes[key]['level'] = 1

            filtered_graph.nodes[key]['interval'] = f"<[{0},{timepoints + 10}]>"

            if not(filtered_graph.nodes[key].get('perturbed')):
                filtered_graph.nodes[key]['perturbed'] = False


    splits = 10
    if timepoints > splits:
        partitions = timepoints // splits
    else:
        partitions = 2
    count = 0
    for key in perturbed_edges:
        if key in filtered_graph:
            if key in normal_edges:
                filtered_graph.nodes[key]['color'] = highlight_color
                filtered_graph.nodes[key]['level'] = 2
            elif key in activator_edges:
                filtered_graph.nodes[key]['color'] = highlight_color
                filtered_graph.nodes[key]['level'] = 3
            else:
                filtered_graph.nodes[key]['color'] = repressor_color
                filtered_graph.nodes[key]['level'] = 4
                

            
            if time_based and not(filtered_graph.nodes[key].get('perturbed')):
                filtered_graph.nodes[key]['color_dynamic'] = f"<[{0},{count},{'#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])}],[{count}, {timepoints},{highlight_color}]>"
                filtered_graph.nodes[key]['perturbed_dynamic'] = f"<[{0},{count},{'#%02x%02x%02x' % tuple(int(255 * c) for c in node_colors[key][:3])}],[{count}, {timepoints},{highlight_color}]>"
                filtered_graph.nodes[key]['color'] = highlight_color
            else:
                if sign_dict.get(key) == 1:
                    filtered_graph.nodes[key]['color'] = highlight_color
                else:
                    filtered_graph.nodes[key]['color'] = repressor_color

            if not(filtered_graph.nodes[key].get('perturbed')):
                filtered_graph.nodes[key]['perturbed'] = True

        count += 1

    ncount = 0
    for nn in move_list:
        if nn in filtered_graph:
            filtered_graph.nodes[nn]['x'] = 2000 + (ncount * 10)
            filtered_graph.nodes[nn]['y'] = 2000 + (ncount * 10)
            if nn in filtered_graph:
                filtered_graph.nodes[nn]['color'] = initiator_color
            
            filtered_graph.nodes[nn]['size'] = max(filtered_graph.nodes[nn].get('size', 0), 50)

            if not(filtered_graph.nodes[nn].get('perturbed')):
                filtered_graph.nodes[nn]['perturbed'] = True

        ncount += 1

        # if count % partitions == 0:
        #     nx.write_graphml(filtered_graph, f"{savepath}/final_graphs/combination_perturbed_third_order_time_based_tp_{count}.graphml")


    return filtered_graph

In [ ]:
import networkx as nx
import json

def export_graph_as_json(G, file_path):
    ll = [ ]
    for n in G.nodes():
        nodeattrs = G.nodes[n]
        genename = str(nodeattrs.get('genename', "NA"))
        if 'genename' in nodeattrs:
            del nodeattrs['genename']
        if genename == "NA" or None:
            genename = gene_id_name_map.get(str(n), str(n))
        ll.append({"data": {"id": str(n), "genename": genename, **nodeattrs}})

    # Convert to Cytoscape.js format
    cy_data = {
        "nodes": ll,
        "edges": [
            {"data": {"id": f"{u}-{v}", "source": str(u), "target": str(v), **G.edges[u, v]}} for u, v in G.edges()
        ]
    }

    # Save to JSON
    with open(file_path, "w") as f:
        json.dump(cy_data, f, indent=2)


In [ ]:
celltype = 'all_cells'
current_edgelist = A_matrix_edgelist
savepath = f'{HWG_BASE_PATH}/combinations/{celltype}'

start_index = 9000
end_index = 10000

perturbed_edges=[]
node_attrs=[]
move_list = []

# filtered_edgelist, filtered_nodes = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)
filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)
filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=False, move_list=move_list)

nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

In [ ]:

# 
# celltype = 'fibroblast'
celltype = 'myotube'
# celltype = 'esc'
# celltype = 'all_cells'
# current_edgelist = A_matrix_edgelist
# current_edgelist = fibroblast_filtered_edgelist
current_edgelist = myotube_filtered_edgelist
# current_edgelist = esc_filtered_edgelist

savepath = f'{HWG_BASE_PATH}/combinations/{celltype}'
graph_json_path = './visualization/data/'

start_index = 6000
end_index = 7000
time_based = False

import json
# with open('/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/connected_A_matrix.json') as f:
# with open(f'/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'{HWG_BASE_PATH}/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
with open(f'./graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'./graphs/combination_perturbed_150_all_cells_{6000}_{7000}.json') as f:
    data = json.load(f)
node_attrs = data['nodes']



perturbed_edges = []
move_list = []


# base network

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, [])
filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)

nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")

base_myotube_network = filtered_graph.copy()

In [ ]:


import matplotlib.cm as cm




celltype = 'fibroblast'
# celltype = 'myotube'
# celltype = 'esc'
# celltype = 'all_cells'
# current_edgelist = A_matrix_edgelist
current_edgelist = fibroblast_filtered_edgelist
# current_edgelist = myotube_filtered_edgelist
# current_edgelist = esc_filtered_edgelist

savepath = f'{HWG_BASE_PATH}/combinations/{celltype}'
graph_json_path = './visualization/data/'

start_index = 5000
end_index = 15000
time_based = False

import json
# with open('/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/connected_A_matrix.json') as f:
# with open(f'/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'{HWG_BASE_PATH}/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'./graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
with open(f'./graphs/combination_perturbed_150_all_cells_{6000}_{7000}.json') as f:
    data = json.load(f)
node_attrs = data['nodes']

# perturbed_edges = []
# move_list = []

# ENSG00000129152 - MYOD1
perturbed_edges = ['ENSG00000129152']
move_list = ['ENSG00000129152']

# mycn, nanog ,pou5f1
# perturbed_edges = ['ENSG00000134323', 'ENSG00000111704', 'ENSG00000204531']
# move_list = ['ENSG00000134323', 'ENSG00000111704', 'ENSG00000204531']

# yamanaka factors
# oct4 - POU5F1
# POU5F1, SOX2, KLF4, MYC
# ['POU5F1',  'KLF4', 'MYC', 'SOX2']
# move_list =  ['ENSG00000204531',  'ENSG00000136826', 'ENSG00000136997', 'ENSG00000181449']


# base network

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, [])
filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)

nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")

base_fibroblast_network = filtered_graph.copy()



# perturbed_edges = []

# first order interactions

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)


# filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, perturbed_edges, node_attrs)
# # nx.write_gexf(filtered_graph, f"{savepath}/combination_connected_{threshold}_{celltype}_start_end_{start_index}_{end_index}.gexf")
# nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_first_order_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)
# nx.write_gexf(filtered_graph, f"{savepath}/combination_connected_{threshold}_{celltype}_start_end_{start_index}_{end_index}.gexf")
nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_first_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_first_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")
first_order_fibroblast_myod_network = filtered_graph.copy()




# second order interactions

exists = (filtered_edgelist['source'].isin(perturbed_edges) | (filtered_edgelist['target'].isin(perturbed_edges)))

matching_rows = filtered_edgelist[exists]
perturbed_edges = list(pd.unique(matching_rows[['source', 'target']].values.ravel()))
print(len(perturbed_edges), perturbed_edges)

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)

# filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, perturbed_edges, node_attrs)
# nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_second_order_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)
nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_second_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_second_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")

second_order_fibroblast_myod_network = filtered_graph.copy()




#third order interactions
exists = (filtered_edgelist['source'].isin(perturbed_edges) | (filtered_edgelist['target'].isin(perturbed_edges)))

matching_rows = filtered_edgelist[exists]
perturbed_edges = list(pd.unique(matching_rows[['source', 'target']].values.ravel()))
print(len(perturbed_edges), perturbed_edges)

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, perturbed_edges)

# filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, perturbed_edges, node_attrs)
# nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_third_order_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)
nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_third_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_third_order_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")

third_order_fibroblast_myod_network = filtered_graph.copy()
# import pandas as pd

# node_data = [
#     {"Id": node, "x": data.get("x", 0), "y": data.get("y", 0)}
#     for node, data in filtered_graph.nodes(data=True)
# ]

# pd.DataFrame(node_data).to_csv(f"{savepath}/combination_perturbed_node_info_{threshold}_{celltype}_start_end_{start_index}_{end_index}.csv", index=False)


# edgelist_df = nx.to_pandas_edgelist(filtered_graph)

# edgelist_df.to_csv(f"{savepath}/combination_connected_{threshold}_{celltype}_start_end_{start_index}_{end_index}.csv", index=False)



In [5]:
import pandas as pd
from constants import DATA_DIRECTORY 

BP_matrix = pd.read_csv(f'{DATA_DIRECTORY}/HWG/BP_Matrix.csv', index_col=0)


BP_matrix

,ENSG00000205609,ENSG00000139579,ENSG00000009307,ENSG00000175329,ENSG00000131910,ENSG00000178028,ENSG00000198538,ENSG00000141052,ENSG00000168724,ENSG00000130254,...,ENSG00000186812,ENSG00000143947,ENSG00000123636,ENSG00000186103,ENSG00000167232,ENSG00000018408,ENSG00000099326,ENSG00000140262,ENSG00000227124,ENSG00000132773
ENSG00000112312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000182968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000007264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000113595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000170004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000206106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000206107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000231068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000243627,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
count = (BP_matrix == 1).sum().sum()
print("Activator counts: ", count)

count = (BP_matrix == -1).sum().sum()
print("Repressor counts: ", count)

count = (BP_matrix == 0).sum().sum()
print("Unknown counts: ", count)

Activator counts:  2563
Repressor counts:  1646
Unknown counts:  48752335


In [ ]:

# 
celltype = 'fibroblast'
# celltype = 'myotube'
# celltype = 'esc'
# celltype = 'all_cells'
# current_edgelist = A_matrix_edgelist
current_edgelist = fibroblast_filtered_edgelist
# current_edgelist = myotube_filtered_edgelist
# current_edgelist = esc_filtered_edgelist

savepath = f'{HWG_BASE_PATH}/combinations/{celltype}'
graph_json_path = './visualization/data/'

start_index = 6000
end_index = 7000
time_based = False

import json
# with open('/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/connected_A_matrix.json') as f:
# with open(f'/Users/ramprakash/development/lab_projects/Rajapakse_lab/projects/HardWiredGenome/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'{HWG_BASE_PATH}/graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
with open(f'./graphs/combination_perturbed_150_all_cells_{start_index}_{end_index}.json') as f:
# with open(f'./graphs/combination_perturbed_150_all_cells_{6000}_{7000}.json') as f:
    data = json.load(f)
node_attrs = data['nodes']



perturbed_edges = []
move_list = []


# base network

filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges = get_piece_network(current_edgelist, A_matrix_edgelist, start_index, end_index, flat_col_order, [])
filtered_graph = construct_graph(filtered_edgelist, filtered_nodes, normal_edges, activator_edges, repressor_edges, perturbed_edges, node_attrs, time_based=time_based, move_list=move_list)

nx.write_graphml(filtered_graph, f"{savepath}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.graphml")

export_graph_as_json(filtered_graph, f"{graph_json_path}/combination_perturbed_base_time_based_{threshold}_{celltype}_start_end_{start_index}_{end_index}.json")

base_myotube_network = filtered_graph.copy()

In [86]:
s_edges = fibroblast_filtered_edgelist[fibroblast_filtered_edgelist['source'] == 'ENSG00000129152']
t_edges = fibroblast_filtered_edgelist[fibroblast_filtered_edgelist['target'] == 'ENSG00000129152']

print(s_edges)
print(t_edges)

Empty DataFrame
Columns: [source, target, weight]
Index: []
Empty DataFrame
Columns: [source, target, weight]
Index: []


In [117]:
threshold = 0
# path = HARD_WIRED_GENOME_A_MATRIX_PATH.strip('.csv') + f"_{threshold}.csv"

# path =  HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}.csv"

# A_matrix_edgelist = pd.read_csv(path, index_col=0)

# full_edgelist = A_matrix_edgelist
# full_edgelist = fibroblast_filtered_edgelist



activator_edges = ['ENSG00000129152']


ts_perturbed_edgelist = full_edgelist[
        full_edgelist['source'].isin(activator_edges) & full_edgelist['target'].isin(flat_col_order)
]
print(ts_perturbed_edgelist)

st_perturbed_edgelist = full_edgelist[
        full_edgelist['source'].isin(flat_col_order) & full_edgelist['target'].isin(activator_edges)
]
print(st_perturbed_edgelist)

perturbed_edgelist = pd.concat([ts_perturbed_edgelist, st_perturbed_edgelist], ignore_index=True)


perturbed_edgelist['edge'] = perturbed_edgelist.apply(lambda row: tuple(sorted([row['source'], row['target']])), axis=1)

df_unique = perturbed_edgelist.drop_duplicates('edge')

df_unique[['source', 'target']] = pd.DataFrame(df_unique['edge'].tolist(), index=df_unique.index)
df = df_unique[['source', 'target']]

mask = df['target'].isin(activator_edges) & ~df['source'].isin(activator_edges)

df.loc[mask, ['source', 'target']] = df.loc[mask, ['target', 'source']].values




Empty DataFrame
Columns: [source, target, weight]
Index: []
Empty DataFrame
Columns: [source, target, weight]
Index: []


ValueError: Columns must be same length as key

In [80]:
myod_targets = list(set(df['target']))
original_potential_targets = len(myod_targets)

Total Myod Targets (all Cell Types) : 1612
Existing Activated Genes (Fibroblast) : 325
Newly MYOD activated Genes (Fibroblast) : 1287


In [112]:
from constants import HUMAN_TF_IDLIST_PATH

def get_TF_set():
    tflist = []
    with open(HUMAN_TF_IDLIST_PATH, 'r') as f:
        for line in f.readlines():
            tflist.append(line.strip())
            
    print(f'processed {len(tflist)} Transcription Factors')
    return tflist

tfset = get_TF_set()
print(tfset[:10])

processed 1639 Transcription Factors
['DUX1_HUMAN', 'DUX3_HUMAN', 'ENSG00000001167', 'ENSG00000004848', 'ENSG00000005073', 'ENSG00000005102', 'ENSG00000005513', 'ENSG00000005801', 'ENSG00000005889', 'ENSG00000006047']


In [122]:
c_matrix = C_matrix

master_regulators = c_matrix[c_matrix['source'] == c_matrix['target']]

master_regulator_list = list(master_regulators['source'])

transcription_factor_matx = c_matrix[c_matrix['source'] != c_matrix['target']]

transcription_factor_list = pd.unique(transcription_factor_matx[['source', 'target']].values.ravel())


In [125]:
master_regulator_set = set(master_regulator_list)
tf_set = set(transcription_factor_list)

In [128]:
fibset = set(fibroblast_expressed_genes)
targetset = set(myod_targets)


print(f'Total Myod Targets (all Cell Types) : {len(targetset)}')
# print(f'Total Expressed Genes in fibroblast : {len(fibset)}')
print(f'Existing Activated Genes (Fibroblast) : {len(targetset.intersection(fibset))}')
print(f'Newly MYOD activated Genes (Fibroblast) : {len(targetset - targetset.intersection(fibset))}')

print('--------------------------')
print(f'Master Regulators within targets : {len(targetset.intersection(master_regulator_set))}')
print(f'Transcription Factors within targets : {len(targetset.intersection(tf_set) - master_regulator_set)}')
print(f'Normal Genes within targets : {len(targetset - tf_set)}')

Total Myod Targets (all Cell Types) : 1612
Existing Activated Genes (Fibroblast) : 325
Newly MYOD activated Genes (Fibroblast) : 1287
--------------------------
Master Regulators within targets : 52
Transcription Factors within targets : 414
Normal Genes within targets : 1146


In [135]:
def classify_node(row):
    if row["Gene ID"] in master_regulator_set:
        return "g1"
    elif row["Gene ID"] in tf_set:
        return "g2"
    else:
        return "g3"

def add_name(row):
    return gene_id_name_map.get(row["Gene ID"], "NA")

In [143]:
fib_G = nx.from_pandas_edgelist(fibroblast_filtered_edgelist, 'source', 'target', edge_attr='weight', create_using=nx.Graph())

fib_G.add_nodes_from(list(targetset))


degree_centrality = nx.degree_centrality(fib_G)
eigen_centrality = nx.eigenvector_centrality(fib_G, max_iter=1000)

df_deg = pd.DataFrame(degree_centrality.items(), columns=["Gene ID", "Degree Centrality"])
df_eig = pd.DataFrame(eigen_centrality.items(), columns=["Gene ID", "Eigenvector Centrality"])

df_all = pd.merge(df_deg, df_eig, on="Gene ID")
df_all = df_all.round(4)

df_all["Gene Type"] = df_all.apply(classify_node, axis=1)
df_all["Gene Name"] = df_all.apply(add_name, axis=1)


df_targets = df_all[df_all["Gene ID"].isin(targetset)]

In [144]:
df_targets.to_csv('./data/myod_first_order_interaction_targets.csv')

In [145]:
df_all

,Gene ID,Degree Centrality,Eigenvector Centrality,Gene Type,Gene Name
0,ENSG00000063978,0.1605,0.0259,g3,RNF4
1,ENSG00000142937,0.1766,0.0397,g3,RPS8
2,ENSG00000198721,0.0940,0.0178,g3,ECI2
3,ENSG00000117118,0.1875,0.0358,g3,SDHB
4,ENSG00000183722,0.0275,0.0023,g3,LHFPL6
...,...,...,...,...,...
3666,ENSG00000177606,0.0000,0.0000,g1,JUN
3667,ENSG00000135457,0.0000,0.0000,g2,TFCP2
3668,ENSG00000187642,0.0000,0.0000,g3,PERM1
3669,ENSG00000141086,0.0000,0.0000,g3,CTRL


In [161]:
print("Degree Centrality", "Eigenvector Centrality")
cnt = 10
top_degsh =  list(df_all.sort_values(by="Degree Centrality", ascending=False)[:cnt]['Gene Name'])
top_egsh = list(df_all.sort_values(by="Eigenvector Centrality", ascending=False)[:cnt]['Gene Name'])
for i in range(cnt):
    # print(top_degsh[i], top_egsh[i])
    print(f"{top_degsh[i]:<20} {top_egsh[i]:<25}")

Degree Centrality Eigenvector Centrality
GAPDH                GAPDH                    
ACTB                 ACTB                     
AKT1                 HSP90AA1                 
HSP90AA1             HSP90AB1                 
HSP90AB1             HSPA4                    
CTNNB1               HSPA8                    
HSPA4                EEF2                     
HSPA8                HSPA5                    
HSPA5                AKT1                     
RHOA                 HSPA9                    


In [149]:
for i in :
    print(i)

GAPDH
ACTB
HSP90AA1
HSP90AB1
HSPA4
HSPA8
EEF2
HSPA5
AKT1
HSPA9


In [154]:
len(fibroblast_expressed_genes)

2638

In [156]:
len(master_regulator_set)

132